# Projekt PRIAD

## Rozpoznawanie autora utworu literackiego na podstawie treści utworu z wykorzystaniem worka słów.

### Stanisław Maliński, Jan Stachurski

Importy

In [2]:
import os
import pandas as pd
import numpy as np

Wczytywanie danych

In [39]:
data_path = "./data/"
file_list = os.listdir(data_path)
func = lambda  a : a.replace(".txt", "").split("-")
count_size = lambda a : os.stat(a).st_size
data = [[func(title)[0], func(title)[1], data_path + title, count_size(data_path + title)] for title in file_list]
files = pd.DataFrame(data, columns=['Author', 'Name of The work', 'FilePath', 'FileSize'])

Przedstawiam zestaw uczący. Mamy tutaj pozycję kilku Polskich autorów.

In [4]:
print(files.drop(columns=['Name of The work', 'FilePath']).groupby('Author').sum())

                   FileSize
Author                     
AdamMickiewicz       761572
HenrykSienkiewicz    867390
Juliusz Słowacki     329618


Wczytanie stop-słów:

In [5]:
def get_stop_words():
    delimeters = ["\\n", "'"]
    stop_words = []
    with open("./stopyPL.txt", encoding="utf8") as f:
        for line in f:
            word = repr(line)
            for c in delimeters:
                word = word.replace(c, "")
            stop_words.append(word)
    return stop_words

Projekt ten używa worka słów, który to przechowuje informacje na temat częstości występowania słów w danym dokumencie. Definiujemy tu dwie funckje jedna do standaryzacji słów tzn. usunięcia ewentualnych znaków graficznych takich jak kropka czy przecinek oraz ustawienie wielkości liter na małe aby słowa "Który" oraz "który" były zliczane jako to same słowo.
Druga funkcja służy do budowania worka słów. Zlicza najpierw pojedyńcze przypadki wystąpień, a następnie zlicza częstość wystąpień danego słowa.

In [13]:
def normalize_word(word):
    word = word.lower()
    special_char = [".", ",", "-", "?", "(", ")", "!", "\\", "\"", ":", ";", "*"]
    for char in special_char:
        word = word.replace(char, "")
    return word

# 0 - generete bag with all words, 1 - bag without stopwords, 2 - bag with only stopwords
def generate_word_bag(FileName, mode=0):
    stop_word = get_stop_words()
    word_bag = {}
    words_in_bag = 0
    with open(FileName, "r", encoding="utf8") as f:
        for line in f:
            words = line.split()
            words = [normalize_word(word) for word in words]
            for word in words:
                if word != '':
                    if (mode==0) or (mode==1 and not word in stop_word) or (mode==2 and word in stop_word):
                        words_in_bag += 1
                        if word in word_bag.keys():
                            word_bag[word]['Count'] += 1
                        else:
                            word_bag[word] = {'Count':1, 'Frequent':1, 'Is stop word': word in stop_word}
                                    
        f.close()
    for word in word_bag.keys():
        word_bag[word]['Frequent'] = word_bag[word]['Count'] / words_in_bag
    return word_bag                                                          

In [14]:
file = "./data/AdamMickiewicz-Burza.txt"
dictionary = generate_word_bag(file)
slowa = dictionary.keys()
slowo = list(slowa)[2]

print("W pliku: \"{}\" Słowo: \"{}\" występowało z częstością: {:.2e}".format(file, slowo, dictionary[slowo]['Frequent']))

W pliku: "./data/AdamMickiewicz-Burza.txt" Słowo: "burza" występowało z częstością: 9.62e-03


In [15]:
def head(dictionary, start=0, end=None, step=1, fname=None):
    words = list(dictionary.keys())
    if end is None:
        end = min(5, len(words))
    if fname is not None:
        print("W pliku: \"{}\"".format(fname))
    for word in words[start:end:step]:
        print("Słowo: \"{}\" występowało z częstością: {:.2e}".format(word,dictionary[word]['Frequent']), sep="")

In [22]:
file = df["FilePath"][1]
dictionary = generate_word_bag(file, 2)
head(dictionary, end=10, fname="data/AdamMickiewicz-Dziady.txt")

W pliku: "data/AdamMickiewicz-Dziady.txt"
Słowo: "w" występowało z częstością: 2.07e-01
Słowo: "już" występowało z częstością: 3.45e-02
Słowo: "z" występowało z częstością: 1.72e-01
Słowo: "się" występowało z częstością: 1.38e-01
Słowo: "i" występowało z częstością: 1.72e-01
Słowo: "jak" występowało z częstością: 6.90e-02
Słowo: "nad" występowało z częstością: 3.45e-02
Słowo: "a" występowało z częstością: 6.90e-02
Słowo: "jego" występowało z częstością: 3.45e-02
Słowo: "dla" występowało z częstością: 3.45e-02


In [23]:
def show_most_frequent(dic ,threshold=0):
    po_czestosci = dict(sorted(dic.items(), key=lambda x: int(x[1]['Count']), reverse = True))
    for index, word in enumerate(po_czestosci.keys()):
        if threshold == 0 and index < 10:
            print(word + " -> " + str(po_czestosci[word]['Count']))
        elif threshold == 0:
            break
        elif po_czestosci[word]['Count'] > threshold:
            print(word + " -> " + str(po_czestosci[word]['Count']))
        else:
            break
show_most_frequent(dictionary)

w -> 6
z -> 5
i -> 5
się -> 4
jak -> 2
a -> 2
już -> 1
nad -> 1
jego -> 1
dla -> 1


Mamy już mały warsztat narzędzi do analizy utworów. Stworzymy teraz parę funckji, które pozwolą nam zgrabnie manipulować macierzą danych. 
merge_dics_to_df - łączy worki słów w ramkę danych, którą później wykorzystamy do nauki klasyfikatora.
tf_idf - analiza TF_IDF.

In [56]:
def merge_dics_to_df(dics, labels=None, transform=lambda x:x):
    tab = []
    word_found = []
    for dic in dics:
        for word in dic.keys():
            if not word in word_found:
                word_found.append(word)
                rec = []
                for dictionary in dics:
                    if word in dictionary.keys():
                        rec.append(transform(dictionary[word]))
                    else:
                        rec.append(0)
                tab.append(rec)
    
    if labels is not None: 
        df = pd.DataFrame(tab, columns=labels, index=word_found)
    else:
        df = pd.DataFrame(tab)
        
    return df.T
    
df = merge_dics_to_df([{'1': 1, '2': 2}, 
                        {'3': 3, '2': 2}, 
                        {'3': 3, '2': 2, '4': 4, '7':7}], 
                       ["Pierwszy słownik", "Drugi słownik", "Trzeci słownik"])
print(df)

                  1  2  3  4  7
Pierwszy słownik  1  2  0  0  0
Drugi słownik     0  2  3  0  0
Trzeci słownik    0  2  3  4  7


In [57]:
def tf_idf(data):   
    ile_slow = data.count(0)[0]
    tf = np.empty((ile_slow,len(data.columns)))
    for i in range(0,len(data.columns)):
        tf[:,i] = np.array(data.iloc[:,i]/data.sum()[i])
                           
    #print((data>0).head())

    #print(np.array((data>0)*1).sum(axis=1))
    
    idf = np.log(len(data.columns)/np.array((data>0)*1).sum(axis=1))

    data['w ilu'] = (np.array((data>0)*1).sum(axis=1)).T
    data['idf'] = idf.T
    
tf_idf(df)
print(df)

                  1  2  3  4  7  w ilu       idf
Pierwszy słownik  1  2  0  0  0      2  0.916291
Drugi słownik     0  2  3  0  0      2  0.916291
Trzeci słownik    0  2  3  4  7      4  0.223144


In [58]:
def drop_if(data, idf_threshold):
    data = data[data.idf <= idf_threshold]
    return data

print(drop_if(df, 1))

                  1  2  3  4  7  w ilu       idf
Pierwszy słownik  1  2  0  0  0      2  0.916291
Drugi słownik     0  2  3  0  0      2  0.916291
Trzeci słownik    0  2  3  4  7      4  0.223144


In [59]:
def drop_if(data, idf_threshold):
    data = data[data.idf <= idf_threshold]
    return data

print(drop_if(df, 1))

                  1  2  3  4  7  w ilu       idf
Pierwszy słownik  1  2  0  0  0      2  0.916291
Drugi słownik     0  2  3  0  0      2  0.916291
Trzeci słownik    0  2  3  4  7      4  0.223144


Teraz musimy stworzyć model, który będzie klasyfikował do jakiego autora należy dany utwór.

Wczytanie obliczenie słowników dla wszystkich utworów.

In [65]:
word_bags = []
index = []
labels = []

for path, name, label in zip(files['FilePath'], files['Name of The work'], files['Author']):
    word_bags.append(generate_word_bag(path, mode=2))
    index.append(name)
    labels.append(label)
    
df = merge_dics_to_df(word_bags, index, lambda x:x['Frequent'])
df['label'] = labels
                                               

In [66]:
df.head()

,na,jak,to,w,się,o,i,za,sobą,lecz,...,bowiem,poza,dużo,dokąd,możliwe,gdziekolwiek,powinno,acz,nia,label
Ajudah,0.129032,0.096774,0.064516,0.129032,0.096774,0.064516,0.096774,0.064516,0.064516,0.032258,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AdamMickiewicz
Ałuszta w dzień,0.034483,0.068966,0.000000,0.206897,0.137931,0.000000,0.172414,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AdamMickiewicz
Ałuszta w nocy,0.130435,0.043478,0.000000,0.086957,0.130435,0.000000,0.130435,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AdamMickiewicz
Bajdary,0.083333,0.083333,0.000000,0.166667,0.083333,0.000000,0.111111,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AdamMickiewicz
Bakczysaraj,0.105263,0.000000,0.052632,0.105263,0.052632,0.105263,0.263158,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AdamMickiewicz


Jak widzimy mamy już ramkę danych, w której każdym obiektem jest dzieło a wektorem atrybutów jest częstość wystąpień poszczególnych słów w każdym z dzieł. Dodaliśmy również atrybuty decyzyjny czyli nazwisko autora danego dzieła.